# Pregunta 24
Optimizar trabajos de PySpark para el rendimiento 
Necesitas aplicar las mejores prácticas y cambios de configuración para optimizar el rendimiento de los trabajos de PySpark.

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName('Pregunta 24').getOrCreate()

df = spark.read.csv("./datasets/employees.csv",header=True,inferSchema=True)
small_df = df # solo es ilustrativo, ya que no realizamos ninguna operacion aqui

In [ ]:
# 1. REPARTITION: Reorganiza datos por columna clave
#    - Agrupa datos relacionados en los mismos nodos
#    - Reduce shuffle en operaciones posteriores (join, groupBy)
#    - Úsalo ANTES de operaciones costosas
df_repart = df.repartition("key_column")

# 2. CACHE: Guarda resultado en memoria RAM
#    - Evita recalcular operaciones repetidas
#    - Mejora rendimiento si reutilizas el DataFrame
#    - Libera memoria con .unpersist() cuando termines
df_cached = df_repart.cache()

# 3. BROADCAST JOIN: Envía DataFrame pequeño a todos los nodos
#    - Evita shuffle del DataFrame grande
#    - Ideal cuando uno de los DataFrames cabe en memoria
#    - Mejora significativa para joins asimétricos
df_joined = df_cached.join(F.broadcast(small_df), "id")

# Resultado: Menos shuffle → Menos movimiento de datos → ⚡ MÁS RÁPIDO